In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [5]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    price = soup.find("span", attrs={'class':'a-price-whole'}).get_text()
    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available



In [6]:

if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=buy+smartphone&crid=3UK1NSVYBWKNQ&sprefix=buy+smartphone%2Caps%2C507&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [7]:
amazon_df

,title,price,rating,reviews,availability
0,Itel P40 (6000mAh Battery with Fast Charging |...,"6,499.",4.5 out of 5 stars,420 ratings,In stock
1,"Oppo F23 5G (Bold Gold, 8GB RAM, 256GB Storage...","24,999.",Previous page,,In stock
2,"Samsung Galaxy M04 Light Green, 4GB RAM, 64GB ...","8,499.",3.9 out of 5 stars,"4,823 ratings",In stock
3,"Nokia C12 Android 12 (Go Edition) Smartphone, ...","5,999.",3.1 out of 5 stars,437 ratings,In stock
4,Samsung Galaxy S21 FE 5G Smartphone Olive,"32,890.",2.9 out of 5 stars,11 ratings,Only 1 left in stock
5,"(Renewed) Xiaomi 11 Lite NE 5G (Jazz Blue, 8GB...","19,499.",3.5 out of 5 stars,7 ratings,Only 1 left in stock.
6,Blackberry Corporation 32GB KEYone 4G LTE Sing...,"23,999.",3.8 out of 5 stars,186 ratings,Only 1 left in stock
7,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM...","18,990.",4.2 out of 5 stars,"124,257 ratings",In stock
8,"Nokia C21 Plus Android Smartphone, Dual SIM, 3...","9,470.",4.0 out of 5 stars,430 ratings,In stock
9,"Nokia C12 Android 12 (Go Edition) Smartphone, ...","5,999.",3.1 out of 5 stars,437 ratings,In stock
